# Airline Passenger Satisfaction
<img src="https://drive.google.com/uc?id=1HvDJElliYQKbdyiCQXsoKdjC8KefRoSL" alt="aitplane in the skies" style="width:500px;"/>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import ipydatagrid
import seaborn as sns
import bqplot as bqp

## I. Boarding (Introduction/background):

Tired of the constant flight delays? Confused about how to book your flight ticket online? Fed up with the mediocre Food and Drink service offered on-board? Perhaps, on the contrary, you are happy with the comfort of your seat, the spaciousness of the leg room, or the variety of in-flight entertainment provided by your airline. Which of these factor(s) matter to you the most, the consumer?
Look no further! We have the answers right here for you!



In our analysis of Airline Passenger Satisfaction, we look at various, different variables affecting customer satisfaction, namely Age, Flight Distance, Departure Delay, Arrival Delay, Departure and Arrival Time Convenience, Ease of Online Booking, Check-in Service, Online Boarding, Gate Location, On-board Service, Seat Comfort, Leg Room Service, Cleanliness, Food and Drink, In-flight Service, In-flight Wifi Service, In-flight Entertainment, and Baggage Handling, with the ultimate goal of trying to determine and understand the factors that contribute most to customer satisfaction.
In our expedition through Exploratory Data Analysis (EDA), we examine, for example, the different data types and the idiosyncratic nature of our attributes. Utilizing assorted charts and graphs, we can visually analyze the distribution of our dataset and scrutinize closely the relationships among the different variables. By uncovering and interpreting patterns in our dataset during EDA, it will help us understand potential risks or trends the models will create in the Modeling phase. 
In the Modeling phase, we use a classification model, namely Logistic Regression, to finally make a prediction about the factors that matter most to customer satisfaction. We will consider different logistic regression techniques, compare their results, and use the most effective one for our model. 
Excited to embark on this data exploratory journey? So are we! Now sit back, relax, and enjoy the flight! Satisfaction guaranteed!

Before we “take off” on our data exploration flight, we import our airline_passenger_satisfaction.csv file, which can be located on Kaggle’s website: https://www.kaggle.com/datasets/mysarahmadbhat/airline-passenger-satisfaction

In [ ]:
airlines_df = pd.read_csv("airline_passenger_satisfaction.csv")

## II. Take-Off (Exploratory Data Analysis)

Now, as our flight glides on the runway and climbs into the sky of EDA, we take a quick look at the individual features, their Non-Null Count (to discover if there are any NULLs in the dataset), and their data type, which is useful later on when we need to manipulate our dataset.

In [ ]:
# Quick overview of the columns and their types
airlines_df.info()

Of course, no data analysis is complete without checking the summary statistics of our dataset. It summarizes the central tendency, dispersion shape of a dataset’s distribution, excluding any NaN values. 

In [ ]:
# Summary statistics
airlines_df.describe()

Earlier, when we looked at the Non-Null Count of each feature, we noticed that the “Arrival Delay column had some NULL values. Let’s look at it using a heatmap!

In [ ]:
# Checking for NaN values
plt.figure(figsize=(16,9))
plt.title("Null values in the dataset")
ax = sns.heatmap(airlines_df.isna().astype(int), cmap='Blues');
ax.set_xlabel("Feature")
ax.set_ylabel("Row");

We sum up all the NULL value for each column and we noticed that there are 393 NULL values.

In [ ]:
airlines_df.isnull().sum()

We look at the “Arrival Delay” column in detail, focusing in on the instances where we have NaN values. Given that we have 393 NULL instances out of 129,880 instances, which is only 0.30%, we decided that the immateriality of the NULL instances warrant dropping those rows altogether so that we can solely focus on the rows with complete information.

In [ ]:
# Examine data with null values
airlines_df[airlines_df['Arrival Delay'].isnull()]

<span style="font-weight: bold;">Given we have only 393 rows with NaN for Arrival Delay, we can probably drop all NaN rows without much impact on the overall analysis</span>

In [ ]:
airlines_df.dropna(inplace=True)

Next, as we settle in on our flight, we look at the age distribution by gender. We can see that the distribution of the data between male and female is very similar in our dataset. 

In [ ]:
# Distribution of ages by gender
# plt.title("Age distribution by gender")
fig = sns.displot(airlines_df, x='Age', kind='hist', col='Gender', kde=True, height=8, aspect=1)
fig.fig.suptitle('Age distribution by gender');

In [ ]:
# Age by bracket
def get_age_bracket(age):
    if age <= 14:
        return "<=14"
    elif age >= 15 and age <= 24:
        return "15-24"
    elif age >=25 and age <= 64:
        return "25-64"
    else:
        return ">65"
airlines_df['Age_Bracket'] = airlines_df["Age"].apply(get_age_bracket)
sns.displot(airlines_df, x='Age_Bracket', kind='hist', col='Gender', kde=False, height=8, aspect=1);

Next, since we were just looking at the count of each gender by age (and by age bracket), we look at the spread for the Satisfaction feature by age. We noticed that there are a lot more younger people who are “Neutral or Dissatisfied” with airlines, while there are slightly more older people who are “Satisfied” with airlines, based on the scores of the various features that were evaluated. 
We definitely have a lot of work to do in the airline industry to satisfy and appeal to the younger folks!

In [ ]:
# Relationship between satisfaction and age
plt.title("Satisfaction distribution by age")
sns.boxplot(data=airlines_df, x="Satisfaction", y="Age", palette="rainbow");

We now plot Satisfaction by age bracket, using the same binning strategy as we used above. Using binning allows us to determine that there is actually a lot of work to do for the airline industry across all age brackets when it comes to customer satisfaction!

In [ ]:
# Relationship between satisfaction and age bracket
# sns.countplot(data=airlines_df, x="Satisfaction")
fig = sns.displot(airlines_df, x='Satisfaction', kind='hist', col='Age_Bracket', kde=False, hue="Satisfaction");
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.fig.suptitle('Satisfaction distribution by age bracket');

This plot confirms our understanding: it shows that we have a bit more “Neutral or Dissatisfied” customers than “Satisfied” Customers. But this is still a relatively balanced sample.

In [ ]:
# Balance between satisfied and unsatisfied customers
plt.rcParams['font.size'] = '16'
fix, ax = plt.subplots(figsize=(16,9))
ax.axes = airlines_df['Satisfaction'].value_counts(normalize=True).plot(kind="barh", color=['salmon', 'dodgerblue'])
ax.set_xlabel("Percentage of total")
ax.set_ylabel("Satisfaction")
plt.title("Balance between satisfied and unsatisfied or neutral");

We next create a Feature Correlation Matrix, which allows us to evaluate the direction as well as the strength of a relationship between variables. Look at the huge correlation between Departure Delay vs Arrival Delay!

In [ ]:
mask = np.zeros_like(airlines_df.corr(), dtype=np.bool_)
mask[np.triu_indices_from(mask)]= True

plt.figure(figsize=(16,9))
plt.rcParams['font.size'] = '12'
plt.title("Feature Correlation Matrix", fontsize=20)
sns.heatmap(airlines_df.corr(), annot=True, mask=mask, linewidths=.5, vmin=-1, vmax=1, fmt=".2f", cmap='coolwarm');

Don’t get out of your seats just yet! The seatbelt sign is still on! We turn on the In-flight entertainment system to view our Interactive EDA

## Interactive EDA

In [ ]:
# Setting up a datagrid
grid = ipydatagrid.DataGrid(airlines_df)

# Some theming to add some color
cotton_candy = {
    "background_color": "rgb(255, 245, 251)",
    "header_background_color": "rgb(207, 212, 252, 1)",
    "header_grid_line_color": "rgb(0, 247, 181, 0.9)",
    "vertical_grid_line_color": "rgb(0, 247, 181, 0.3)",
    "horizontal_grid_line_color": "rgb(0, 247, 181, 0.3)",
    "selection_fill_color": "rgb(212, 245, 255, 0.3)",
    "selection_border_color": "rgb(78, 174, 212)",
    "header_selection_fill_color": "rgb(212, 255, 239, 0.3)",
    "header_selection_border_color": "rgb(252, 3, 115)",
    "cursor_fill_color": "rgb(186, 32, 186, 0.2)",
    "cursor_border_color": "rgb(191, 191, 78)",
}

grid.grid_style = cotton_candy

In [ ]:
# Define widgets and variables
numerical_cols = airlines_df.select_dtypes([int, float]).columns.tolist()
age_range = (airlines_df['Age'].min(), airlines_df['Age'].max())
columns_index = {k:grid._column_name_to_index(k) + 1 for k in airlines_df.columns}
range_slider = widgets.IntRangeSlider(min=age_range[0], max=age_range[1], value=age_range, description="Age Range")
gender_dropdown = widgets.Dropdown(options=airlines_df['Gender'].unique().tolist() + ["Both"], 
                                   value="Both", layout={"width":"200px"}, description="Gender")
scatter_x_dropdown = widgets.Dropdown(options=numerical_cols, 
                                      value=numerical_cols[2], layout={"width":"250px"}, description="Scatter X-Axis")
scatter_y_dropdown = widgets.Dropdown(options=numerical_cols, value=numerical_cols[1], 
                                      layout={"width":"250px"}, description="Scatter Y-Axis")

# Chart
sc_x = bqp.LinearScale()
sc_y = bqp.LinearScale()
scatt = bqp.ScatterGL(
    x=airlines_df[scatter_x_dropdown.value].values,
    y=airlines_df[scatter_y_dropdown.value].values,
    names=np.arange(10),
    scales={"x": sc_x, "y": sc_y},
    colors=["limegreen", "purple"],
)
ax_x = bqp.Axis(scale=sc_x, label=scatter_x_dropdown.value)
ax_y = bqp.Axis(scale=sc_y, orientation="vertical", tick_format="d", label=scatter_y_dropdown.value)
fig = bqp.Figure(marks=[scatt], axes=[ax_x, ax_y], padding_x=0.025, interaction=bqp.interacts.PanZoom(scales={'x': [sc_x], 'y': [sc_y]}),
                 title="Select axes from the dropdown boxes!")


# Event handlers
def filter_gender(e):
    with grid.hold_sync():
        selected_gender = e.get("new")
        if selected_gender == "Both":
            grid._transforms = list(filter(lambda x: x['columnIndex'] != columns_index.get("Gender"), grid._transforms))
            return 

        grid.transform([
            {"type": "filter", "operator": "=", "columnIndex": columns_index.get("Gender"), "value": selected_gender},
            {'type': 'filter', 'columnIndex': columns_index.get("Age"), 'operator': 'between', 'value': range_slider.value}
        ])
        update_scatter_chart(None)
    
def filter_age(e):
    with grid.hold_sync():
        age_tuple = e.get("new")
        grid.transform([
            {'type': 'filter', 'columnIndex': columns_index.get("Age"), 'operator': 'between', 'value': age_tuple}
        ])
        update_scatter_chart(None)
    
def update_scatter_chart(e):
    with scatt.hold_sync():
        data = grid.get_visible_data()
        scatt.x = data[scatter_x_dropdown.value].values
        scatt.y = data[scatter_y_dropdown.value].values
        sc_x.min = float(data[scatter_x_dropdown.value].min())
        sc_x.max = float(data[scatter_x_dropdown.value].max())
        sc_y.min = float(data[scatter_y_dropdown.value].min())
        sc_y.max = float(data[scatter_y_dropdown.value].max())
        ax_x.label = scatter_x_dropdown.value
        ax_y.label = scatter_y_dropdown.value
        fig.title = f"{scatter_x_dropdown.value} vs. {scatter_y_dropdown.value}"
    
    
gender_dropdown.observe(filter_gender, names=['value'])
range_slider.observe(filter_age, names=['value'])
scatter_x_dropdown.observe(update_scatter_chart, names=["value"])
scatter_y_dropdown.observe(update_scatter_chart, names=["value"])


# Rendering the grid and widgets
widgets.VBox([
    widgets.HTML(value="<h1>Airline Passengers Data Explorer</h1>"),
    widgets.HBox([
        range_slider, gender_dropdown, scatter_x_dropdown, scatter_y_dropdown
    ], layout=widgets.Layout(flex='1 1 auto', width='100%')),
    grid,
    fig
])

## III. In-Flight (Modeling)

In [ ]:
from sklearn.model_selection import train_test_split

<span style="font-weight: bold;">Before we start train/test splitting, we need to reason about the kind of analysis we want to do, and whether the data, in its current form, is suitable.
The obvious target for this dataset, given we're dealing with passenger satisfaction with a given airline, would be to to predict whether a potential
passenger would be satisfies or neutral/unsatisfied given some paramaters like their age, sex, travel class, flight delay etc.</span>

<span style="font-weight: bold;">This type of analysis lends itself nicely to Logistic Regression, which could be the ideal model to use for this analysis, with this dataset.
To use logistic regression, we are likely going to need to One-Hot Encode categorical series so they're represented as numbers.</span>

In [ ]:
airlines_df.info()

<span style="font-weight: bold;">It's possible to see that we have quite a few columns which contain categorical data, with them ost crucial column being the predicted column.</span>

In [ ]:
# Columns with categorical data
airlines_df.select_dtypes(["object"]).head(3)

<span style="font-weight: bold;">Below, we create a new dataframe which has all categorical values encoded to integers. We will use that dataframe to train and test our models.</span>

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Function to retrieve the string to integer mapping used in One-Hot encoding
def get_encoding_mapping(encoded_series, types):
    return dict(zip(encoded_series.classes_, encoded_series.transform(types)))

# Creating a copy of the airlines dataframe, as we will encode and replace 
# categorical columns with their numerical equivalents
airlines_df_encoded = airlines_df.copy()

# Mapping dictionary to convert back from encoding to string - will be used later!
enc_mappings = []

for col in airlines_df_encoded.select_dtypes(["object"]).columns:
    # One-Hot encoding categorical columns
    cur_series = airlines_df_encoded[col]
    col_enc = LabelEncoder()
    col_enc.fit(cur_series)
    col_unique_vals = cur_series.unique().tolist()
    enc_mappings.append(get_encoding_mapping(col_enc, col_unique_vals))
    airlines_df_encoded[col] = col_enc.transform(cur_series)

In [ ]:
# Checking the conversion succeeded
airlines_df_encoded.info()

In [ ]:
# The 'ID' columns is unlikely to be useful for our model, so we will drop it.
airlines_df_encoded = airlines_df_encoded.drop('ID', axis=1)

<span style="font-weight: bold;">Our data is now in the right shape to be used as an input to a ML model! Recall that some columns had very high positive/negative correlation. We should keep an eye on those and potentially drop some of them as inputs so that we reduce the amount of multicollinearity in the model.</span>

In [ ]:
# Columns used to predict "Satisfaction"
feature_list = list(filter(lambda x: x != 'Satisfaction', airlines_df_encoded.columns.tolist()))

In [ ]:
# Define regressor and regressand
y = airlines_df_encoded[['Satisfaction']]
X = airlines_df_encoded[feature_list]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Sanity check everything looks okay
display(X_train.head(3))

<span style="font-weight: bold;">We're ready to instantiate and fit our model now! But before we do that, it would be good to define a helper function which can standardise the way we fit, train and score our models. This will reduce code duplication and the potential for operational.</span>

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay, accuracy_score, roc_auc_score, classification_report

def run_model(model, X_train, X_test, y_train, y_test, print_results=True):
    # Convert to 1D arrays as some of the sklearn functions require them    
    y_train = y_train.values.ravel()
    y_test = y_test.values.ravel()
    
    # Fit our and train our model     
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred) 
    cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
    if print_results:
        print(f"Accuracy:{accuracy}, ROC AUC: {roc_auc}")
        print(classification_report(y_test,y_pred,digits=5))
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 7))
        cmd = ConfusionMatrixDisplay.from_estimator(model, X_test, y_test, ax=ax1)
        rocd = RocCurveDisplay.from_estimator(model, X_test, y_test, ax=ax2)
    
    return model, accuracy, roc_auc

<span style="font-weight: bold;">We can also add a function for inspecting the regression coefficients for LogisticRegression, so we can get more insight about the relevance of our features. This function is defined below.</span>

In [ ]:
import statsmodels.api as sm

def infer_logistic_regression_coefs(y_train, X_train):
    logit_model = sm.Logit(y_train, X_train)
    result = logit_model.fit()
    print(result.summary())

#### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Running our model (using a higher number of iteration to maximize chance of model convergence)
lg_model, lg_accuracy, lg_roc_auc = run_model(LogisticRegression(max_iter=4000), X_train, X_test, y_train, y_test);

<span style="font-weight: bold;">Some additional statistics. Looking into the regression model and determining which features are important and affect our regression.</span>

In [ ]:
infer_logistic_regression_coefs(y_train, X_train)

<span style="font-weight: bold;">Let's add regularization. We will start with L1 ratio of 1, which means we're looking at full Lasso regression. In scikit-learn, ElsticNet regularized logistic regression must be performed using the "saga" solver.</span>

In [ ]:
model, accuracy, roc_auc = run_model(LogisticRegression(penalty="elasticnet", l1_ratio=0.5, solver="saga"), X_train, X_test, y_train, y_test);

<span style="font-weight: bold;">We were able to use the 'saga' solver in order to add a regularization term, but we can see the <span style='color:red'>warning</span> indicating the solver wasn't able to converge. This is because logistic regression models with regularization are not scale invariant! We can improve the proformance of the 'saga' solver by standardizing the input. Let's do that below!</span>

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

model, accuracy, roc_auc = run_model(Pipeline([('scaler', StandardScaler()), ('logistic', LogisticRegression(penalty="elasticnet", l1_ratio=1, solver="saga"))]), X_train, X_test, y_train, y_test);

<span style="font-weight: bold;">The solver is able to converge and we're getting a better result compared to the cell above. But the score is roughly the same as the score of the model without regularization. Let's run a grid search to see if tweaking the l1 ratio can help up achieve better performance.</span>

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {"logistic__l1_ratio": np.arange(0,1.1,0.1)}

gcv = GridSearchCV(estimator=Pipeline([('scaler', StandardScaler()), ('logistic', LogisticRegression(penalty="elasticnet", solver="saga"))]), param_grid=parameters, n_jobs=4)
gcv.fit(X_train, y_train.values.ravel())
print(f"Best L1 regularization rario parameter: {gcv.best_params_}")

<span style="font-weight: bold;">It turns out that our chosen parameter L1 ratio parameter is the best choice for the model. In other words, a full Lasso regression seems to perform better than a mix of, or only Ridge regression.</span>

#### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

model, accuracy, roc_auc = run_model(GaussianNB(), X_train, X_test, y_train, y_test);

#### K-Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model, accuracy, roc_auc = run_model(KNeighborsClassifier(n_neighbors=10, algorithm="kd_tree", n_jobs=4), X_train, X_test, y_train, y_test);

#### Decision Tree Classifer

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model, accuracy, roc_auc = run_model(DecisionTreeClassifier(max_depth=5, random_state=0, max_leaf_nodes=10), X_train, X_test, y_train, y_test, True);

We can visualize the tree

In [ ]:
from sklearn import tree
plt.figure(figsize=(16,9), dpi=120)
tree.plot_tree(model, label='all', feature_names=feature_list, fontsize=10);

#### Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier

model, accuracy, roc_auc = run_model(MLPClassifier(), X_train, X_test, y_train, y_test, True);

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model, accuracy, roc_auc = run_model(RandomForestClassifier(), X_train, X_test, y_train, y_test, True);

#### Adaptive Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model, accuracy, roc_auc = run_model(AdaBoostClassifier(), X_train, X_test, y_train, y_test, True);

## V: Arrival

We have reached the end of our flight towards data exploration and modeling. 
We hope you had an enjoyable journey. Godspeed. 

## IV: Landing (Description of challenges/obstacles faced)

We have reached the end of our flight towards data exploration and modeling. 
We hope you had an enjoyable journey. Godspeed. 